In [16]:
jobs = jobs_manager()

In [ ]:
cache = set()
chunks = list(log_progress(list_chunk_sizes(), every=1))
for chunk in chunks:
    for record in log_progress(load_chunk(chunk.name), every=10000, size=chunk.size):
        cache.add(record.selection)

In [207]:
selections= [_ for _ in generate_selections() if _ not in cache]

In [ ]:
for index, start in enumerate(xrange(0, 31695, 3000)):
    print "%job dump_catalog_records(log_progress(fetch_selections(log_progress(selections[{start}:{stop}], every=1)), every=10), '{index}')".format(
        start=start,
        stop=start+3000,
        index=index+29
    )

In [119]:
size = sum(_.size for _ in chunks)
records = load_chunks(_.name for _ in chunks)
stream = map_names(log_progress(records, every=10000, size=size))
write_table(stream, NAMES_TABLE)

In [ ]:
sort_table(NAMES_TABLE, by=NAME_COLUMN, chunks=40)

In [212]:
size = get_table_size(NAMES_TABLE)

In [213]:
stream = read_table(NAMES_TABLE)
groups = group_stream(log_progress(stream, every=100000, size=size), by=NAME_COLUMN)
stream = reduce_names(groups)
write_table(stream, UNIQUE_NAMES_TABLE)

In [ ]:
sort_rn_table(UNIQUE_NAMES_TABLE)

In [226]:
size = get_table_size(UNIQUE_NAMES_TABLE)

In [227]:
records = load_unique_names(UNIQUE_NAMES_TABLE)
records = (_ for _ in log_progress(records, every=10000, size=size) if parse_name(_.name) and _.count > 1)
stream = serialize_unique_names(records)
write_table(stream, FILTERED_UNIQUE_NAMES_TABLE)